In [1]:
import requests
import cohere
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import openai
import nltk
from textblob import TextBlob
from nltk.tokenize import word_tokenize, sent_tokenize
import torch

In [18]:
import pyphen

In [19]:
dic = pyphen.Pyphen(lang='en')

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/manasverma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     /Users/manasverma/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [3]:
# API keys
ai21_api_key = 'your-api-key'
cohere_api_key = 'your-api-key'
huggingface_api_key = 'your-api-key'
openai_api_key = 'your-api-key'

In [4]:
def generate_text_ai21(prompt):
    url = "https://api.ai21.com/studio/v1/j2-light/complete"
    headers = {
        "Authorization": f"Bearer {ai21_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "prompt": prompt,
        "numResults": 1,
        "maxTokens": 50,
        "temperature": 0.7,
        "topKReturn": 0,
        "topP": 1,
        "stopSequences": []
    }
    response = requests.post(url, headers=headers, json=data)
    result = response.json()
    return result['completions'][0]['data']['text']

In [5]:
def generate_text_cohere(prompt):
    co = cohere.Client(cohere_api_key)
    response = co.generate(
        model='command-xlarge-nightly',
        prompt=prompt,
        max_tokens=50,
        temperature=0.7,
        k=0,
        p=1,
        stop_sequences=[]
    )
    return response.generations[0].text.strip()

In [6]:
def generate_text_huggingface(prompt):
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

    tokenizer.pad_token = tokenizer.eos_token

    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=150,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=2.0
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [7]:
def generate_text_openai(prompt):
    openai.api_key = openai_api_key
    response = openai.Completion.create(
        engine="davinci-002",
        prompt=prompt,
        max_tokens=50,
        temperature=0.7,
    )
    return response.choices[0].text.strip()

In [8]:
#grammar checks

In [9]:
def basic_grammar_check(text):
    words = word_tokenize(text)
    spell = nltk.corpus.words.words()
    misspelled = [word for word in words if word.lower() not in spell]
    return len(misspelled), misspelled

In [21]:
def flesch_reading_ease(text):
    sentences = len(sent_tokenize(text))
    words = len(word_tokenize(text))
    syllables = sum([count_syllables(word) for word in word_tokenize(text)])
    if sentences == 0 or words == 0:
        return 0
    asl = words / sentences
    asw = syllables / words
    score = 206.835 - (1.015 * asl) - (84.6 * asw)
    return score

In [20]:
def count_syllables(word):
    return len(dic.inserted(word).split('-'))

In [11]:
def count_tokens(text):
    tokens = word_tokenize(text)
    return len(tokens)

In [12]:
def count_sentences(text):
    sentences = sent_tokenize(text)
    return len(sentences)

In [13]:
def average_word_length(text):
    words = word_tokenize(text)
    return sum(len(word) for word in words) / len(words)

In [22]:
prompt = "Places to visit in summer"

In [23]:
# AI21 Studio Evaluation
try:
    ai21_result = generate_text_ai21(prompt)
    ai21_grammar_issues_count, ai21_grammar_issues = basic_grammar_check(ai21_result)
    ai21_readability = flesch_reading_ease(ai21_result)
    ai21_token_count = count_tokens(ai21_result)
    ai21_sentence_count = count_sentences(ai21_result)
    ai21_avg_word_length = average_word_length(ai21_result)
    print(f"AI21 Studio:\nGenerated Text: {ai21_result}\nGrammar Issues: {ai21_grammar_issues_count}\nReadability Score: {ai21_readability}\nToken Count: {ai21_token_count}\nSentence Count: {ai21_sentence_count}\nAverage Word Length: {ai21_avg_word_length}\n")
except Exception as e:
    print(f"Error evaluating AI21 Studio: {e}")

AI21 Studio:
Generated Text: 

Summer is the perfect time to go on a vacation or weekend getaway. It's also a great time explore new places and try new things. If you're looking for some ideas, here are a few places to visit in the summer:

1. Beaches: Beaches are a great choice for a summer vacation, as they offer plenty of sun, sand, and ocean breeze.
2
Grammar Issues: 20
Readability Score: 91.21800000000002
Token Count: 75
Sentence Count: 5
Average Word Length: 3.506666666666667



In [24]:
# Cohere Evaluation
try:
    cohere_result = generate_text_cohere(prompt)
    cohere_grammar_issues_count, cohere_grammar_issues = basic_grammar_check(cohere_result)
    cohere_readability = flesch_reading_ease(cohere_result)
    cohere_token_count = count_tokens(cohere_result)
    cohere_sentence_count = count_sentences(cohere_result)
    cohere_avg_word_length = average_word_length(cohere_result)
    print(f"Cohere:\nGenerated Text: {cohere_result}\nGrammar Issues: {cohere_grammar_issues_count}\nReadability Score: {cohere_readability}\nToken Count: {cohere_token_count}\nSentence Count: {cohere_sentence_count}\nAverage Word Length: {cohere_avg_word_length}\n")
except Exception as e:
    print(f"Error evaluating Cohere: {e}")

Cohere:
Generated Text: Here is a list of places to visit for a summer vacation:
1. Paris: Paris in the summer is a popular choice for tourists. The weather is warm, and the city offers a wealth of cultural attractions, including museums, monuments, and
Grammar Issues: 18
Readability Score: 71.12605442176874
Token Count: 49
Sentence Count: 3
Average Word Length: 3.857142857142857



In [25]:
# Hugging Face Evaluation
try:
    huggingface_result = generate_text_huggingface(prompt)
    huggingface_grammar_issues_count, huggingface_grammar_issues = basic_grammar_check(huggingface_result)
    huggingface_readability = flesch_reading_ease(huggingface_result)
    huggingface_token_count = count_tokens(huggingface_result)
    huggingface_sentence_count = count_sentences(huggingface_result)
    huggingface_avg_word_length = average_word_length(huggingface_result)
    print(f"Hugging Face:\nGenerated Text: {huggingface_result}\nGrammar Issues: {huggingface_grammar_issues_count}\nReadability Score: {huggingface_readability}\nToken Count: {huggingface_token_count}\nSentence Count: {huggingface_sentence_count}\nAverage Word Length: {huggingface_avg_word_length}\n")
except Exception as e:
    print(f"Error evaluating Hugging Face: {e}")

Hugging Face:
Generated Text: Places to visit in summer.


"This is a very beautiful place, I will be back soon," he said with confidence and dignity while standing beside the water at their new home for about 10 people on Friday morning."My husband was born there last year - it's been my life since that time so when we moved here from Mumbai two years ago (I'm living alone) this city has also given me an opportunity", Prakash told NDTV after his family arrived today as part of 'The Year' festival organised by Dhananathaya Nagar Bhavan under section 2(2)(a).Pankaj Gupta says she went out looking forward because her house had recently come down hit hard which meant even more money needed
Grammar Issues: 40
Readability Score: 55.73040767386094
Token Count: 139
Sentence Count: 3
Average Word Length: 3.964028776978417



In [26]:
# OpenAI Evaluation
try:
    openai_result = generate_text_openai(prompt)
    openai_grammar_issues_count, openai_grammar_issues = basic_grammar_check(openai_result)
    openai_readability = flesch_reading_ease(openai_result)
    openai_token_count = count_tokens(openai_result)
    openai_sentence_count = count_sentences(openai_result)
    openai_avg_word_length = average_word_length(openai_result)
    print(f"OpenAI:\nGenerated Text: {openai_result}\nGrammar Issues: {openai_grammar_issues_count}\nReadability Score: {openai_readability}\nToken Count: {openai_token_count}\nSentence Count: {openai_sentence_count}\nAverage Word Length: {openai_avg_word_length}\n")
except Exception as e:
    print(f"Error evaluating OpenAI: {e}")

OpenAI:
Generated Text: in the United States

People often plan trips to the US in summer as they look to have a good time away from their regular lives. There are different places they can visit, depending on the things that they want to do. Here are some of
Grammar Issues: 8
Readability Score: 90.11789115646262
Token Count: 49
Sentence Count: 3
Average Word Length: 3.857142857142857

